In [ ]:
# libraries
import numpy as np
import pandas as pd 
import torch
import random
from torch.nn.utils.rnn import pad_sequence
from torchmetrics.functional import pearson_corrcoef
from torchmetrics import Metric
import torch.nn as nn
from torch.utils.data import Dataset
from transformers import Trainer
from sklearn.model_selection import train_test_split
import itertools

In [ ]:
# model functions
class CorrCoef(Metric):
    def __init__(self):
        super().__init__()
        self.add_state("corrcoefs", default=torch.tensor(0.0), dist_reduce_fx="sum")
        self.add_state("total", default=torch.tensor(0.0), dist_reduce_fx="sum")
    def update(self, preds, target, mask):
        preds = preds[:, :, 1] # second head is deprivation full prediction
        assert preds.shape == target.shape
        assert preds.shape == mask.shape
        coeffs = []
        for p, t, m in zip(preds, target, mask):
            mp, mt = torch.masked_select(p, m), torch.masked_select(t, m)
            temp_pearson = pearson_corrcoef(mp, mt)
            coeffs.append(temp_pearson)
        coeffs = torch.stack(coeffs)
        self.corrcoefs += torch.sum(coeffs)
        self.total += len(coeffs)
    def compute(self):
        return self.corrcoefs / self.total

# collate function
def collate_fn(batch):
    # batch is a list of tuples (x, y)
    x, y, ctrl_y, gene, transcript = zip(*batch)

    # sequence lenghts 
    lengths = torch.tensor([len(x) for x in x])
    
    x = pad_sequence(x, batch_first=True, padding_value=192) 
    y = pad_sequence(y, batch_first=True, padding_value=-1)
    ctrl_y = pad_sequence(ctrl_y, batch_first=True, padding_value=-1)

    out_batch = {}

    out_batch["input_ids"] = x
    out_batch["labels"] = y
    out_batch["lengths"] = lengths
    out_batch["labels_ctrl"] = ctrl_y

    return out_batch

# compute metrics
def compute_metrics(pred):
    labels = pred.label_ids 
    preds = pred.predictions
    inputs = pred.inputs
    mask = labels != -100.0
    labels = torch.tensor(labels)
    preds = torch.tensor(preds)
    preds = torch.squeeze(preds, dim=2)
    mask = torch.tensor(mask)
    # mask = torch.arange(preds.shape[1])[None, :].to(lengths) < lengths[:, None]
    mask = torch.logical_and(mask, torch.logical_not(torch.isnan(labels)))
    corr_coef = CorrCoef()
    corr_coef.update(preds, labels, mask)

    return {"r": corr_coef.compute()}

# compute metrics
def compute_metrics_saved(pred):
    '''
    additional function to just save everything to do analysis later
    '''
    labels = pred.label_ids 
    preds = pred.predictions
    inputs = pred.inputs
    mask = labels != -100.0
    labels = torch.tensor(labels)
    preds = torch.tensor(preds)
    preds = torch.squeeze(preds, dim=2)
    mask = torch.tensor(mask)
    # mask = torch.arange(preds.shape[1])[None, :].to(lengths) < lengths[:, None]
    mask = torch.logical_and(mask, torch.logical_not(torch.isnan(labels)))
    corr_coef = CorrCoef()
    corr_coef.update(preds, labels, mask)

    # save predictions
    preds = preds.cpu().numpy()
    labels = labels.cpu().numpy()

    np.save("preds/preds.npy", preds)
    np.save("preds/labels.npy", labels)
    np.save("preds/inputs.npy", inputs)

    return {"r": corr_coef.compute()}

In [ ]:
# global variables
id_to_codon = {idx:''.join(el) for idx, el in enumerate(itertools.product(['A', 'T', 'C', 'G'], repeat=3))}
codon_to_id = {v:k for k,v in id_to_codon.items()}

# dataset generation functions
def longestZeroSeqLength(a):
    '''
    length of the longest sub-sequence of zeros
    '''
    a = a[1:-1].split(', ')
    a = [float(k) for k in a]
    # longest sequence of zeros
    longest = 0
    current = 0
    for i in a:
        if i == 0.0:
            current += 1
        else:
            longest = max(longest, current)
            current = 0
    return longest

def percNans(a):
    '''
    returns the percentage of nans in the sequence
    '''
    a = a[1:-1].split(', ')
    a = [float(k) for k in a]
    a = np.asarray(a)
    perc = np.count_nonzero(np.isnan(a)) / len(a)

    return perc

def RiboDatasetGWS(data_folder: str, ds: str, threshold: float = 0.6, longZerosThresh: int = 20, percNansThresh: float = 0.1):
    if ds == 'ALL':
        # # paths 
        # ctrl_path = data_folder + 'CTRL.csv'
        # leu_path = data_folder + 'LEU.csv'
        # arg_path = data_folder + 'ARG.csv'

        # # load the data
        # df_ctrl = pd.read_csv(ctrl_path)
        # df_ctrl['condition'] = 'CTRL'
        # df_leu = pd.read_csv(leu_path)
        # df_leu['condition'] = 'LEU'
        # df_arg = pd.read_csv(arg_path)
        # df_arg['condition'] = 'ARG'

        # df_full = pd.concat([df_ctrl, df_leu, df_arg], axis=0) # ctrl, leu, arg

        # # drop first column
        # df_full = df_full.drop(['Unnamed: 0'], axis=1)

        # df_full.columns = ['gene', 'transcript', 'sequence', 'annotations', 'perc_non_zero_annots', 'condition']

        # # apply annot threshold
        # df_full = df_full[df_full['perc_non_zero_annots'] >= threshold]

        # # for all the sequences in a condition that is not CTRL, add their respective CTRL sequence to them
        # sequences_ctrl = []
        # annotations_list = list(df_full['annotations'])
        # condition_df_list = list(df_full['condition'])
        # transcripts_list = list(df_full['transcript'])

        # for i in range(len(condition_df_list)):
        #     try:
        #         if condition_df_list[i] != 'CTRL':
        #             # find the respective CTRL sequence for the transcript
        #             ctrl_sequence = df_full[(df_full['transcript'] == transcripts_list[i]) & (df_full['condition'] == 'CTRL')]['annotations'].iloc[0]
        #             sequences_ctrl.append(ctrl_sequence)
        #         else:
        #             sequences_ctrl.append(annotations_list[i])
        #     except:
        #         sequences_ctrl.append('NA')

        # # add the sequences_ctrl to the df
        # print(len(sequences_ctrl), len(annotations_list))
        # df_full['ctrl_sequence'] = sequences_ctrl

        # # remove those rows where the ctrl_sequence is NA
        # df_full = df_full[df_full['ctrl_sequence'] != 'NA']

        # # sanity check for the ctrl sequences
        # # get the ds with only condition as CTRL
        # df_ctrl_full = df_full[df_full['condition'] == 'CTRL']
        # ctrl_sequences_san = list(df_ctrl_full['annotations'])
        # ctrl_sequences_san2 = list(df_ctrl_full['ctrl_sequence'])

        # for i in range(len(ctrl_sequences_san)):
        #     assert ctrl_sequences_san[i] == ctrl_sequences_san2[i]

        # print("Sanity Checked")

        # # get longest zero sequence length for each sequence in annotations and ctrl_sequence
        # annotations_list = list(df_full['annotations'])
        # sequences_ctrl = list(df_full['ctrl_sequence'])
        # annotation_long_zeros = []
        # ctrl_sequence_long_zeros = []
        # num_nans_full = []
        # num_nans_ctrl = []
        # for i in range(len(annotations_list)):
        #     annotation_long_zeros.append(longestZeroSeqLength(annotations_list[i]))
        #     ctrl_sequence_long_zeros.append(longestZeroSeqLength(sequences_ctrl[i]))
        #     num_nans_full.append(percNans(annotations_list[i]))
        #     num_nans_ctrl.append(percNans(sequences_ctrl[i]))

        # # add the longest zero sequence length to the df
        # df_full['longest_zero_seq_length_annotation'] = annotation_long_zeros
        # df_full['longest_zero_seq_length_ctrl_sequence'] = ctrl_sequence_long_zeros

        # # add the number of nans to the df
        # df_full['perc_nans_annotation'] = num_nans_full
        # df_full['perc_nans_ctrl_sequence'] = num_nans_ctrl

        # # apply the threshold for the longest zero sequence length
        # df_full = df_full[df_full['longest_zero_seq_length_annotation'] <= longZerosThresh]
        # df_full = df_full[df_full['longest_zero_seq_length_ctrl_sequence'] <= longZerosThresh]

        # # apply the threshold for the number of nans
        # df_full = df_full[df_full['perc_nans_annotation'] <= percNansThresh]
        # df_full = df_full[df_full['perc_nans_ctrl_sequence'] <= percNansThresh]

        # # GWS
        # genes = df_full['gene'].unique()
        # genes_train, genes_test = train_test_split(genes, test_size=0.2, random_state=42)

        # # split the dataframe
        # df_train = df_full[df_full['gene'].isin(genes_train)]
        # df_test = df_full[df_full['gene'].isin(genes_test)]

        out_train_path = 'data/dh/ribo_train_ALL-NA_dh_' + str(threshold) + '_NZ_' + str(longZerosThresh) + '_PercNan_' + str(percNansThresh) + '.csv'
        out_test_path = 'data/dh/ribo_test_ALL-NA_dh_' + str(threshold) + '_NZ_' + str(longZerosThresh) + '_PercNan_' + str(percNansThresh) + '.csv'

        # df_train.to_csv(out_train_path, index=False)
        # df_test.to_csv(out_test_path, index=False)

        df_train = pd.read_csv(out_train_path)
        df_test = pd.read_csv(out_test_path)

        return df_train, df_test

class GWSDatasetFromPandas(Dataset):
    def __init__(self, df):
        self.df = df
        self.counts = list(self.df['annotations'])
        self.sequences = list(self.df['sequence'])
        self.condition_lists = list(self.df['condition'])
        self.condition_values = {'CTRL': 0, 'LEU': 1, 'ARG': 2}

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        X = self.df['sequence'].iloc[idx]
        # convert to int
        X = X[1:-1].split(', ')
        X = [int(i) for i in X]

        y = self.df['annotations'].iloc[idx]
        # convert string into list of floats
        y = y[1:-1].split(', ')
        y = [float(i) for i in y]

        y = [1+i for i in y]
        y = np.log(y)

        # ctrl sequence 
        ctrl_y = self.df['ctrl_sequence'].iloc[idx]
        # convert string into list of floats
        ctrl_y = ctrl_y[1:-1].split(', ')
        ctrl_y = [float(i) for i in ctrl_y]

        # no min max scaling
        ctrl_y = [1+i for i in ctrl_y]
        ctrl_y = np.log(ctrl_y)

        X = np.array(X)
        # multiply X with condition value times 64 + 1
        add_factor = (self.condition_values[self.condition_lists[idx]]) * 64
        X += add_factor

        y = np.array(y)
        len_X = len(X)

        X = torch.from_numpy(X).long()
        y = torch.from_numpy(y).float()
        ctrl_y = torch.from_numpy(ctrl_y).float()

        gene = self.df['gene'].iloc[idx]
        transcript = self.df['transcript'].iloc[idx]

        return X, y, ctrl_y, gene, transcript

In [ ]:
# loss functions
class MaskedPearsonLoss(nn.Module):
    def __init__(self):
        super().__init__()
    def __call__(self, y_pred, y_true, mask, eps=1e-6):
        y_pred_mask = torch.masked_select(y_pred, mask)
        y_true_mask = torch.masked_select(y_true, mask)
        cos = nn.CosineSimilarity(dim=0, eps=eps)
        return 1 - cos(
            y_pred_mask - y_pred_mask.mean(),
            y_true_mask - y_true_mask.mean(),
        )

class MaskedL1Loss(nn.Module):
    def __init__(self):
        super().__init__()

    def __call__(self, y_pred, y_true, mask):
        y_pred_mask = torch.masked_select(y_pred, mask).float()
        y_true_mask = torch.masked_select(y_true, mask).float()

        loss = nn.functional.l1_loss(y_pred_mask, y_true_mask, reduction="none")
        return torch.sqrt(loss.mean())

class MaskedNormMAELoss(nn.Module):
    def __init__(self):
        super().__init__()

    def __call__(self, y_pred, y_true, mask):
        y_pred_mask = torch.masked_select(y_pred, mask).float()
        y_true_mask = torch.masked_select(y_true, mask).float()

        loss = nn.functional.l1_loss(y_pred_mask, y_true_mask, reduction="none") 
        # get mean y true without nans
        # convert y_true_mask to numpy
        y_true_mask = y_true_mask.cpu().numpy()
        y_true_mean = np.nanmean(y_true_mask)

        return torch.sqrt(loss.mean()) / y_true_mean

class MaskedCombinedDoubleHeadPearsonLoss(nn.Module):
    def __init__(self, alpha):
        super().__init__()
        self.pearson = MaskedPearsonLoss()
        self.l1 = MaskedL1Loss()
        self.alpha = alpha
    
    def __call__(self, y_pred, labels, labels_ctrl, mask_full, mask_ctrl):
        y_pred_ctrl = y_pred[:, :, 0]
        y_pred_depr_diff = y_pred[:, :, 1]
        y_pred_full = torch.sum(y_pred, dim=2)
        labels_diff = labels - labels_ctrl
        # combine masks to make mask diff 
        mask_diff = mask_full & mask_ctrl

        loss_ctrl = self.pearson(y_pred_ctrl, labels_ctrl, mask_ctrl)
        loss_depr_diff = self.l1(y_pred_depr_diff, labels_diff, mask_diff)
        loss_full = self.pearson(y_pred_full, labels, mask_full)

        return loss_ctrl + (self.alpha*loss_depr_diff) + loss_full

class MaskedCombinedDoubleHeadMAELoss(nn.Module):
    def __init__(self, alpha):
        super().__init__()
        self.l1 = MaskedL1Loss()
        self.alpha = alpha
    
    def __call__(self, y_pred, labels, labels_ctrl, mask_full, mask_ctrl):
        y_pred_ctrl = y_pred[:, :, 0]
        y_pred_depr_diff = y_pred[:, :, 1]
        y_pred_full = torch.sum(y_pred, dim=2)
        labels_diff = labels - labels_ctrl
        # combine masks to make mask diff 
        mask_diff = mask_full & mask_ctrl

        loss_ctrl = self.l1(y_pred_ctrl, labels_ctrl, mask_ctrl)
        loss_depr_diff = self.l1(y_pred_depr_diff, labels_diff, mask_diff)
        loss_full = self.l1(y_pred_full, labels, mask_full)

        return loss_ctrl + (self.alpha*loss_depr_diff) + loss_full

class MaskedCombinedDoubleHeadNormMAELoss(nn.Module):
    def __init__(self, alpha):
        super().__init__()
        self.l1 = MaskedNormMAELoss()
        self.alpha = alpha
    
    def __call__(self, y_pred, labels, labels_ctrl, mask_full, mask_ctrl):
        y_pred_ctrl = y_pred[:, :, 0]
        y_pred_depr_diff = y_pred[:, :, 1]
        y_pred_full = torch.sum(y_pred, dim=2)
        labels_diff = labels - labels_ctrl
        # combine masks to make mask diff 
        mask_diff = mask_full & mask_ctrl

        loss_ctrl = self.l1(y_pred_ctrl, labels_ctrl, mask_ctrl)
        loss_depr_diff = self.l1(y_pred_depr_diff, labels_diff, mask_diff)

        print(loss_ctrl, loss_depr_diff)

        return loss_ctrl + (self.alpha*loss_depr_diff)

class MaskedCombinedDoubleHeadTwoMAEPCCLoss(nn.Module):
    def __init__(self, alpha):
        super().__init__()
        self.l1 = MaskedL1Loss()
        self.pcc = MaskedPearsonLoss()
        self.alpha = alpha
    
    def __call__(self, y_pred, labels, labels_ctrl, mask_full, mask_ctrl):
        y_pred_ctrl = y_pred[:, :, 0]
        y_pred_depr_diff = y_pred[:, :, 1]
        y_pred_full = torch.sum(y_pred, dim=2)
        labels_diff = labels - labels_ctrl
        # combine masks to make mask diff 
        mask_diff = mask_full & mask_ctrl

        loss_ctrl = self.l1(y_pred_ctrl, labels_ctrl, mask_ctrl)
        loss_depr_diff = self.pcc(y_pred_depr_diff, labels_diff, mask_diff)

        return loss_ctrl + loss_depr_diff

class MaskedCombinedDoubleHeadTwoMAELoss(nn.Module):
    def __init__(self, alpha):
        super().__init__()
        self.l1 = MaskedL1Loss()
        self.alpha = alpha
    
    def __call__(self, y_pred, labels, labels_ctrl, mask_full, mask_ctrl):
        y_pred_ctrl = y_pred[:, :, 0]
        y_pred_full = y_pred[:, :, 1]

        loss_ctrl = self.l1(y_pred_ctrl, labels_ctrl, mask_ctrl)
        loss_full = self.l1(y_pred_full, labels, mask_full)

        return loss_ctrl + loss_full

In [ ]:
# custom MAE regression trainer
class RegressionTrainerMAE(Trainer):
    def __init__(self, alpha, **kwargs,):
        super().__init__(**kwargs)
        self.alpha = alpha

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        labels_ctrl = inputs.pop("labels_ctrl")
        outputs = model(**inputs)
        logits = outputs.logits
        logits = torch.squeeze(logits, dim=2)
        lengths = inputs['lengths']

        loss_fnc = MaskedCombinedDoubleHeadMAELoss(self.alpha)
        
        mask_full = torch.arange(labels.shape[1])[None, :].to(lengths) < lengths[:, None]
        mask_full = torch.logical_and(mask_full, torch.logical_not(torch.isnan(labels)))

        mask_ctrl = torch.arange(labels_ctrl.shape[1])[None, :].to(lengths) < lengths[:, None]
        mask_ctrl = torch.logical_and(mask_ctrl, torch.logical_not(torch.isnan(labels_ctrl)))
        
        loss = loss_fnc(logits, labels, labels_ctrl, mask_full, mask_ctrl)

        return (loss, outputs) if return_outputs else loss 

# custom PCC-L1-PCC regression trainer
class RegressionTrainerPCC(Trainer):
    def __init__(self, alpha, **kwargs,):
        super().__init__(**kwargs)
        self.alpha = alpha

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        labels_ctrl = inputs.pop("labels_ctrl")
        outputs = model(**inputs)
        logits = outputs.logits
        logits = torch.squeeze(logits, dim=2)
        lengths = inputs['lengths']

        loss_fnc = MaskedCombinedDoubleHeadPearsonLoss(self.alpha)
        
        mask_full = torch.arange(labels.shape[1])[None, :].to(lengths) < lengths[:, None]
        mask_full = torch.logical_and(mask_full, torch.logical_not(torch.isnan(labels)))

        mask_ctrl = torch.arange(labels_ctrl.shape[1])[None, :].to(lengths) < lengths[:, None]
        mask_ctrl = torch.logical_and(mask_ctrl, torch.logical_not(torch.isnan(labels_ctrl)))
        
        loss = loss_fnc(logits, labels, labels_ctrl, mask_full, mask_ctrl)

        return (loss, outputs) if return_outputs else loss 

# custom MAE on CTRL and PCC on Diff regression trainer
class RegressionTrainerTwoMAEPCC(Trainer):
    def __init__(self, alpha, **kwargs,):
        super().__init__(**kwargs)
        self.alpha = alpha

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        labels_ctrl = inputs.pop("labels_ctrl")
        outputs = model(**inputs)
        logits = outputs.logits
        logits = torch.squeeze(logits, dim=2)
        lengths = inputs['lengths']

        loss_fnc = MaskedCombinedDoubleHeadTwoMAEPCCLoss(self.alpha)
        
        mask_full = torch.arange(labels.shape[1])[None, :].to(lengths) < lengths[:, None]
        mask_full = torch.logical_and(mask_full, torch.logical_not(torch.isnan(labels)))

        mask_ctrl = torch.arange(labels_ctrl.shape[1])[None, :].to(lengths) < lengths[:, None]
        mask_ctrl = torch.logical_and(mask_ctrl, torch.logical_not(torch.isnan(labels_ctrl)))
        
        loss = loss_fnc(logits, labels, labels_ctrl, mask_full, mask_ctrl)

        return (loss, outputs) if return_outputs else loss 

# custom MAE on CTRL and PCC on Diff regression trainer
class RegressionTrainerTwoNormMAE(Trainer):
    def __init__(self, alpha, **kwargs,):
        super().__init__(**kwargs)
        self.alpha = alpha

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        labels_ctrl = inputs.pop("labels_ctrl")
        outputs = model(**inputs)
        logits = outputs.logits
        logits = torch.squeeze(logits, dim=2)
        lengths = inputs['lengths']

        loss_fnc = MaskedCombinedDoubleHeadNormMAELoss(self.alpha)
        
        mask_full = torch.arange(labels.shape[1])[None, :].to(lengths) < lengths[:, None]
        mask_full = torch.logical_and(mask_full, torch.logical_not(torch.isnan(labels)))

        mask_ctrl = torch.arange(labels_ctrl.shape[1])[None, :].to(lengths) < lengths[:, None]
        mask_ctrl = torch.logical_and(mask_ctrl, torch.logical_not(torch.isnan(labels_ctrl)))
        
        loss = loss_fnc(logits, labels, labels_ctrl, mask_full, mask_ctrl)

        return (loss, outputs) if return_outputs else loss 

# custom MAE on CTRL and PCC on Diff regression trainer
class RegressionTrainerTwoMAE(Trainer):
    def __init__(self, alpha, **kwargs,):
        super().__init__(**kwargs)
        self.alpha = alpha

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        labels_ctrl = inputs.pop("labels_ctrl")
        outputs = model(**inputs)
        logits = outputs.logits
        logits = torch.squeeze(logits, dim=2)
        lengths = inputs['lengths']

        loss_fnc = MaskedCombinedDoubleHeadTwoMAELoss(self.alpha)
        
        mask_full = torch.arange(labels.shape[1])[None, :].to(lengths) < lengths[:, None]
        mask_full = torch.logical_and(mask_full, torch.logical_not(torch.isnan(labels)))

        mask_ctrl = torch.arange(labels_ctrl.shape[1])[None, :].to(lengths) < lengths[:, None]
        mask_ctrl = torch.logical_and(mask_ctrl, torch.logical_not(torch.isnan(labels_ctrl)))
        
        loss = loss_fnc(logits, labels, labels_ctrl, mask_full, mask_ctrl)

        return (loss, outputs) if return_outputs else loss 